In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud

pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from fractions import Fraction
from decimal import *
TWOPLACES = Decimal(10) ** -2

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'width'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=702
attribute_id_input=2426

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [5]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
# _______________________________________________________________________________________________

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

# def find_first_digit(s, non=False):
#     for i, x in enumerate(s):
#         if x.isdigit() ^ non:
#             return i
#     return -1

# def split_digits(s, case=False):
#     non = True
#     while s:
#         i = find_first_digit(s, non)
#         if i == 0:
#             non = not non
#         elif i == -1:
#             yield int(s) if s.isdigit() else s if case else s.lower()
#             s = ''
#         else:
#             x, s = s[:i], s[i:]
#             yield int(x) if x.isdigit() else x if case else x.lower()

# def natural_key(s, *args, **kwargs):
#     return tuple(split_digits(s, *args, **kwargs))

# _______________________________________________________________________________________________

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
    
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
        
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))


def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

measure='This is a fraction: 100 1/64'
[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

# #remove duplicates
# matches['matches']=[list(set(sublist)) for sublist in matches['matches']]

['This is a fraction: 100.02']

In [6]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(buckets)]
len(df)

7092

In [7]:
regex_pattern=r'(?i)(\d+.?\d?\/?\d*\\\"\s?W(?!eigh)\b)|(\d+\.\d*\\\"\s?W(?!eigh)\b)|()'
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

In [11]:
regex_pattern_blank=r'\d'
df['new_matches'] = df['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_len=df[df['matches'].astype(str)!='[]']
df_len=df_len[df_len['new_matches'].astype(str)!='[]']

In [13]:
from natsort import natsorted
regex_pattern_comma=r','
df_len['comma_matches'] = df_len['matches'].apply(lambda x: re_extract(regex_pattern_comma, str(x)))
natural_sort=df_len[df_len['comma_matches'].astype(str)!='[]']
natural_sort['matches']=natural_sort['matches'].apply(lambda x: natsorted(x))
# natural_sort['matches']=natural_sort['matches'].apply(lambda x: remove_duplicates(x))
# natural_sort['matches']=[list(set(sublist)) for sublist in natural_sort['matches']]
# natural_sort['matches']= natural_sort['matches'].apply(lambda x: sorted(x, key=natural_keys))
natural_sort['matches'][0:500] 

26                                                                               [3\" W, 26.5\" W]
84                                                                        [13\" W, 16\" W, 56\" W]
85                                                                        [13\" W, 16\" W, 56\" W]
86                                                                        [13\" W, 16\" W, 56\" W]
101      [20\" W, 39\" W, 54\" W, 60\" W, 66\" W, 68\" W, 72\" W, 78\" W, 81\" W, 90\" W, 108\" W]
105      [20\" W, 39\" W, 54\" W, 60\" W, 66\" W, 68\" W, 72\" W, 78\" W, 81\" W, 90\" W, 108\" W]
121      [20\" W, 39\" W, 54\" W, 60\" W, 66\" W, 68\" W, 72\" W, 78\" W, 81\" W, 90\" W, 108\" W]
229                                                                       [12\" W, 14\" W, 16\" W]
1244                                                                              [12\" W, 24\" W]
3696                                                                              [15\" W, 35\" W]
5098      

In [14]:
natural_sort['matches']=natural_sort['matches'].apply(lambda x: re.sub(r'\\" W|"W',' in',str(x))).apply(lambda x: re.sub(r'\\" W|"W|\"xa0W|xa0W|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
# natural_sort['matches']=[list(set(sublist)) for sublist in natural_sort['matches']]
natural_sort['matches']=natural_sort['matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
natural_sort['matches']= natural_sort['matches'].apply(lambda x: sorted(x, key=natural_keys))
natural_sort['matches']=natural_sort['matches'].apply(lambda x: re.sub("\['15.9, 15.9'\]","['15.9']",str(x)))
natural_sort['matches']=natural_sort['matches'].apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub(",",'","',str(x))).apply(lambda x: re.sub('" ','"',str(x))).apply(lambda x: re.sub('","',' in","',str(x))).apply(lambda x: re.sub('"]',' in"]',str(x))).apply(lambda x: re.sub('in in','in',str(x)))                                                                          
# natural_sort['matches']=[list(set(sublist)) for sublist in natural_sort['matches']]
natural_sort['matches'][0:500]

0                                                         ["31 in","31.5 in"]
1                                                         ["23 in","23.6 in"]
3                                                        ["33 in","34.62 in"]
4                                                         ["77.5 in","83 in"]
8                                                          ["1 in","1.25 in"]
17                                                      ["32.5 in","34.5 in"]
50                                                          ["18 in","21 in"]
71                                                          ["13 in","16 in"]
139                                                        ["7.5 in","10 in"]
140                                                     ["8.75 in","15.5 in"]
144                           ["39.5 in","54.5 in","54 in","61.5 in","78 in"]
154                                                     ["22.5 in","23.5 in"]
159                                 ["4.88 in","6.38 in","7.06 i

In [17]:
natural_sort[curation_col]=natural_sort['matches']
matches_natural=natural_sort[['external_id',curation_col]]
matches_natural

external_id                                            Q:width
0        69644361                                ["31 in","31.5 in"]
1        69545447                                ["23 in","23.6 in"]
3        69594953                               ["33 in","34.62 in"]
4        69694930                                ["77.5 in","83 in"]
8        45526367                                 ["1 in","1.25 in"]
...           ...                                                ...
51393    45826290                               ["10 in","39.75 in"]
51407    60167590                                ["11 in","13.5 in"]
51434    46008039  ["13 in","16 in","18 in","22 in","27 in","29 in"]
51443    47230439                                  ["18 in","21 in"]
51467    67148080                            ["17.25 in","19.25 in"]

[1138 rows x 2 columns]

In [ ]:
# def remove_frac(numbers):
#     if str(numbers).find(',') > -1:  
#         nom, den = numbers.split('/')
#         tmp = float(nom)/float(den)
#         return tmp
#     else:
#         return numbers

In [78]:
# df_a = pd.DataFrame({"A": [[16.0, 24.4175], [14.9687, 16.06], [22.75, 23.00]]})
# # df_a = pd.DataFrame({"A": [[16, 24.42], [14.97, 16.06], [22.75, 23]]})
# df_a

In [76]:
# df_a.applymap(lambda x: [round(v, None if v.is_integer() else 2) for v in x])

In [77]:
# x=[16.00, 24.4175]
# [round_string_float(num) for num in x]

In [69]:
# df_a = pd.DataFrame({"A": [[16.0, 24.4175], [14.9687, 16.06], [22.75, 23.00]]})
# df_a.applymap(lambda x: [round(v, None if v.is_integer() else 2) for v in x])

A
0     [16, 24.42]
1  [14.97, 16.06]
2     [22.75, 23]

In [20]:
# natural_sort['matches']=natural_sort['matches'].apply(lambda x: re.sub(r'\\" W|"W|\"xa0W|xa0W|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
# # natural_sort['matches']=natural_sort['matches'].apply(lambda x: remove_frac(remove_fraction(x)))
# # natural_sort['matches']=natural_sort['matches'].apply(lambda x: [round_string_float(num) for num in x])
# x=pd.DataFrame(natural_sort['matches']).applymap(lambda x: [round(v, None if v.is_integer() else 2) for v in x])
# # natural_sort['matches'].apply(lambda x: [round(v, None if float(v).is_integer() else 2) for v in x])
# list = []
# for i in range(len(x)):
#     for j in range(len(x[i])):
#         list .append(round(x[i][j]))
# x

In [10]:
df_new=df_len[df_len['comma_matches'].astype(str)=='[]']
df_new['matches']=df_new['matches'].apply(lambda x: re.sub(r'\\" W|"W|\"xa0W|xa0W|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
df_new['matches']=df_new['matches'].apply(lambda x: remove_frac(remove_fraction(x)))
df_new['matches']=df_new['matches'].apply(lambda x: round_string_float(x))

In [11]:
df_new['matches']=df_new['matches'].apply(lambda x:str(x)+' in')

In [12]:
df_new[curation_col]=df_new['matches']
matches=df_new[['external_id',curation_col]]
matches

external_id  Q:width
1        62866903    18 in
2        69763719    18 in
3        69763717    18 in
5        42595717    13 in
6        61535404     7 in
...           ...      ...
50261    69400100     3 in
50266    66882459  27.5 in
50272    64215396   117 in
50273    61608726   117 in
50274    61608733   117 in

[11652 rows x 2 columns]

In [39]:
# df_length['matches']= df_length['matches'].apply(lambda x: sorted(x, key=natural_key))

In [66]:
# df_length['new_matches']=df_length['matches'].apply(lambda x: re.sub(r'\\".?L','',str(x))).apply(lambda x: re.sub(r"'",'',str(x))).apply(lambda x: re.sub(r'\\','',str(x))).apply(lambda x: re.sub(r'"','',str(x)))       

In [91]:
# df_length['new_matches']= df_length['new_matches'].apply(lambda x: sorted(x, key=natural_key))
# print(len(df_length['new_matches'].explode().value_counts()))
# df_length['new_matches'].explode().value_counts()[1000:1500]

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [18]:
def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-multiples.csv',index=False) 

In [19]:
looks_good('Bed Bath & Beyond', attribute, df, matches_natural)